# JBFG Data Analysis Competition
- Null 처리방식 : 2번

## PC Environment and Library Version
***

In [ ]:
#!pip install watermark
%load_ext watermark
%watermark -a 'DataLine' -nmv --packages numpy,pandas,sklearn,imblearn,tensorflow,plotly,matplotlib,seaborn,missingno

## 탐색적 데이터 분석
***

### Library and Data Loading, Function Definition

#### Import Library for Data Analysis

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl  
import missingno as msno
import warnings

import plotly.express as px
import plotly.graph_objs as go
# import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.offline as pyo
pyo.init_notebook_mode()


warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid")
%matplotlib inline

mpl.rc('font', family='Malgun Gothic')  # 한글 폰트 설정
                                        # 윈도우 폰트 위치 - C:\Windows\Fonts
plt.figure(figsize=(10,6))              # 그래프 사이즈 설정
sns.set(font='Malgun Gothic', rc={'axes.unicode_minus':False}, style='darkgrid') # 마이너스 처리


### Function Definition

#### 연속형 데이터 그래프 함수

In [ ]:
def print_continuous_graphs(df, column, column_desc):

       counts = df[column].value_counts() # 해당 컬럼의 속성별 합계
       exist_counts = df[df['is_churned'] == 0][column].value_counts() # 유지 - 해당 컬럼의 속성별 합계
       churn_counts = df[df['is_churned'] != 0][column].value_counts() # 이탈 - 해당 컬럼의 속성별 합계
       churn_rates = df[df['is_churned'] == 1][column].value_counts() / df[column].value_counts() # 해당 컴럼의 속성별 이탈율    

       fig = make_subplots(rows=3, 
                     cols=2, 
                     subplot_titles=('전체 건수 분포', '유지/이탈별 사분위', '유지/이탈별 분포'), 
                     # shared_xaxes=True,
                     horizontal_spacing=0.1,
                     vertical_spacing=0.1,
                     specs=[[{"secondary_y": True}, {}],
                            [{}, {"secondary_y": True}],
                            [{"secondary_y": True},{}],
                            ]
                     )

       # 전체
       # ----
       fig.add_trace(go.Histogram(x=df[df['is_churned']!=0][column],  marker_color="red", name='이탈'), row=1, col=1, secondary_y=False)
       # fig.add_trace(go.Histogram(x=df[df['is_churned']!=0][column], texttemplate="%{x}", marker_color="red"), row=1, col=1, secondary_y=False)
       
       fig.add_trace(go.Histogram(x=df[df['is_churned']==0][column], marker_color="blue", name='유지'), row=1, col=1, secondary_y=False)
       fig.add_trace(go.Scatter(x=churn_rates.sort_index().index, y=churn_rates.sort_index(), marker_color="green", name='이탈율', line_shape='linear'),
                     row=1, col=1, secondary_y=True)
       fig.update_yaxes(secondary_y=True, range=[0, 1], row=1, col=1)


       
       # Box Graph
       # ---------
       # fig.add_trace(go.Box(x=exist_counts, 
       #               name='유지'), row=2, col=1)
       fig.add_trace(go.Box(x=df[df['is_churned']==0][column].sort_values(), name='유지', marker_color="blue"), row=1, col=2)
       fig.add_trace(go.Box(x=df[df['is_churned']!=0][column].sort_values(), name='이탈', marker_color="red"), row=1, col=2)


       # Histogram Graph
       # ---------------

       fig.add_trace(go.Histogram(x=df[df['is_churned']==0][column], marker_color="blue"), row=2, col=1)
       fig.add_trace(go.Histogram(x=df[df['is_churned']!=0][column], marker_color="red"), row=2, col=1)

       # Scatter Graph
       # -------------
       fig.add_trace(go.Scatter(x=churn_counts.sort_index().index, y=churn_counts.sort_index(), mode='lines+markers', marker_color="red", name='이탈'), row=2, col=2, secondary_y=False)
       fig.add_trace(go.Scatter(x=exist_counts.sort_index().index, y=exist_counts.sort_index(), mode='lines+markers', marker_color='blue', name='유지'), row=2, col=2, secondary_y=False)

       fig.add_trace(go.Scatter(x=churn_rates.sort_index().index, y=churn_rates.sort_index(), marker_color="green", name='이탈율', line_shape='linear'),
                     row=2, col=2, secondary_y=True)


       # 이탈률
       # ------
       # churn_rates = df[df['is_churned'] == 1][column].value_counts() / df[column].value_counts() # 해당 컴럼의 속성별 이탈율    
       fig.add_trace(go.Histogram(x=churn_rates.sort_index()), row=3, col=1)
       fig.update_yaxes(secondary_y=True, range=[0, 1], row=3, col=1)


       fig.update_layout(width=1200, 
                     height=1200, 
                     showlegend=False,
                     barmode='stack',
                     hovermode="x",
                     )

       fig.show()


In [ ]:
def category_func(df, column, start_value, units):
    bins = np.arange(start_value, df[column].max()+units, units)
    bins_label = [str(round(x,2)) for x in bins]
    df[f"{column}_category"] = pd.cut(df[column], bins, right=True, include_lowest=True, labels=bins_label[:-1])

    return df

#eda_churner_df = category_func(eda_churner_df, 'mean_util_pct', 0.2)

#### 범주형 데이터 그래프 함수

In [ ]:
def print_category_graphs(df, column, column_desc):
    
    counts = df[column].value_counts() # 해당 컬럼의 속성별 합계
    exist_counts = df[df['is_churned'] == 0][column].value_counts() # 유지 - 해당 컬럼의 속성별 합계
    churn_counts = df[df['is_churned'] != 0][column].value_counts() # 이탈 - 해당 컬럼의 속성별 합계
    churn_rates = df[df['is_churned'] == 1][column].value_counts() / df[column].value_counts() # 해당 컴럼의 속성별 이탈율    
    
    
    fig = make_subplots(rows=3, 
                    cols=2, 
                    subplot_titles=('【 전체 현황 】', '【 이탈율 】', '【 사분위 】', f'【 {column_desc} 중 전체 현황 】', f'【 {column_desc} 중 유지 현황 】', f'【 {column_desc} 중 이탈 현황 】'), 
                    # shared_xaxes=True,
                    horizontal_spacing=0.1,
                    vertical_spacing=0.1,
                    specs=[[{"secondary_y": True}, {}],
                           [{}, {'type':'domain'}],
                           [{'type':'domain'}, {'type':'domain'}]]
                   )


    # 전체 현황
    # ---------
    fig.add_trace(go.Bar(x=churn_counts.sort_index().index, y=churn_counts.sort_index(), marker_color="red", offsetgroup=0, name='이탈', 
                         text=churn_counts.sort_index(), 
                         hovertemplate = '%{label}: %{value:,}',
                         textposition='auto'), row=1, col=1, secondary_y=False)
    
    
    fig.add_trace(go.Bar(x=exist_counts.sort_index().index, y=exist_counts.sort_index(), marker_color="blue", offsetgroup=0, name='유지', 
                         texttemplate='%{value:,}', 
                        #  text=exist_counts.sort_index(), 
                         hovertemplate = '%{label}: %{value:,}',
                         textposition='auto', base=churn_counts.sort_index()), row=1, col=1, secondary_y=False)
    
    fig.add_trace(go.Scatter(x=churn_rates.sort_index().index, y=churn_rates.sort_index(), marker_color="green", name='이탈율', 
                             line_shape='linear'), row=1, col=1, secondary_y=True)
    
    fig.update_yaxes(secondary_y=True, range=[0, 1], row=1, col=1)
    #fig.update_traces(texttemplate='%{value:,}', hovertemplate = '%{label}, %{value}', row=1, col=1)
    # fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
    

    # 이탈율
    # ------
    fig.add_trace(go.Bar(x=churn_rates.sort_index().index, y=churn_rates.sort_index(), marker_color="red", name='이탈율'),
                  row=1, col=2)

    
    # 사분위
    # ------
    fig.add_trace(go.Box(x=df[df['is_churned']!=0][column].sort_values(), marker_color="red", name='이탈'), row=2, col=1)
    fig.add_trace(go.Box(x=df[df['is_churned']==0][column].sort_values(), marker_color="blue", name='유지'), row=2, col=1)


    # 유지/이탈 현황
    # -------------
    fig.add_trace(go.Pie(labels=counts.sort_index().index, values=counts.sort_index(), name=f'{column_desc} 분표 현황', texttemplate = "%{label}: %{value:,} <br>(%{percent})",
                         textposition = "inside"), row=2, col=2)
    fig.update_traces(hole=.4, hoverinfo="label+percent+name", row=2, col=2)

  
    # 유지 현황
    # ---------
    fig.add_trace(go.Pie(labels=exist_counts.sort_index().index, values=exist_counts.sort_index(), name="유지", texttemplate = "%{label}: %{value:,} <br>(%{percent})",
                         textposition = "inside"), row=3, col=1)
    fig.update_traces(hole=.4, hoverinfo="label+percent+name", row=3, col=1)


    # 이탈 현황
    # ---------
    fig.add_trace(go.Pie(labels=churn_counts.sort_index().index, values=churn_counts.sort_index(), name="이탈", texttemplate = "%{label}: %{value:,} <br>(%{percent})",
                         textposition = "inside"), row=3, col=2)
    fig.update_traces(hole=.4, hoverinfo="label+percent+name", row=3, col=2)


    fig.add_annotation(dict(x=0.73, y=0.5, ax=0, ay=0,
                    xref = "paper", yref = "paper", 
                    text= "<b>전체</b>", 
                    font_size=20,
                  ))

    fig.add_annotation(dict(x=0.21, y=0.13, ax=0, ay=0,
                        xref = "paper", yref = "paper", 
                        text= "<b>유지</b>", 
                        font_size=20,
                      ))

    fig.add_annotation(dict(x=0.73, y=0.13, ax=0, ay=0,
                        xref = "paper", yref = "paper", 
                        text= "<b>이탈</b>", 
                        font_size=20,
                      ))

    
    fig.update_layout(width=1200, 
                  height=1200, 
                  showlegend=False,
                  title_text=f'『 {column_desc} 』에 따른 분석 그래프',
                # barmode='stack'
                  hovermode="x",
                 )
    

    fig.show()    

#### 데이터 개략 확인 함수
- 데이터 건수, null 비율 등으로 변경해서 사용하면 좋을 듯함.

# Calculate the percentages of "Yes" and "No" responses for each education level
education_percents = bank_churner_df_org.groupby('education')['is_churned'].value_counts(normalize=True).unstack()
education_percents.reset_index(inplace=True)
education_percents.fillna(0, inplace=True)  # Fill with 0 for missing values

# Create a DataFrame with the percentages of "No" and "Yes" responses for each Education_Level value
education_percent_table = education_percents.rename(columns={0: 'No_Percentage', 1: 'Yes_Percentage'})

# Format the percentages as percentage notation
education_percent_table['No_Percentage'] = education_percent_table['No_Percentage'].apply(lambda x: f"{x:.2%}")
education_percent_table['Yes_Percentage'] = education_percent_table['Yes_Percentage'].apply(lambda x: f"{x:.2%}")

# Sort the table in descending order based on the "Yes_Percentage" value
education_percent_table = education_percent_table.sort_values(by='Yes_Percentage', ascending=False)

# Print the percentage table without the index column
print(education_percent_table[['education', 'No_Percentage', 'Yes_Percentage']].to_string(index=False))

### Data Loading

In [ ]:
eda_churner_df = pd.read_csv("./data/bank_churner.csv")

### 분석 데이터 개략 확인
***

#### 데이터 세트 정보 확인
- 일부 Feature에 Null 값 존재함을 확인 함
- 향후 모델학습시 Null 값 처리에 대한 필요성 확인 함
- 모델학습을 의해 Oject 항목을 적절하게 변형할 필요성을 확인 함 - sex, education, marital_stat, imcome_cat, card_type (5개 Features) 

In [ ]:
eda_churner_df.info()

#### 수치 데이터의 분포값 개략 확인

In [ ]:
eda_churner_df.describe()

#### 결측치 확인 및 시각화

In [ ]:
print(eda_churner_df.isnull().sum())

null_rates = eda_churner_df.isnull().sum() / 8101 * 100
null_rates


In [ ]:
plt.figure(figsize=(24, 12))
msno.bar(eda_churner_df)
plt.show()

### Feature별 특징 분석

#### is_churned : 이탈  여부
- 0 : 유지, 1 : 이탈

In [ ]:
eda_churner_df["is_churned"].value_counts()

In [ ]:
tot_cnt = eda_churner_df['is_churned'].count().sum()
tot_null_cnt = eda_churner_df['is_churned'].isnull().sum()
print(f'전체 데이터 건수 = {tot_cnt:,} Null 건수 = {tot_null_cnt:,} 전체 데이터 중 널 비율 =  {round(tot_null_cnt / tot_cnt,2)}') 

In [ ]:
fig = go.Figure()
fig.add_trace(go.Pie(labels=['유지', '이탈'], values=eda_churner_df['is_churned'].value_counts().sort_index(), name='이탈별 분포', 
                     texttemplate = "%{value:,}명 <br><b>(%{percent})</b>",
                     title='<b>전체<br> 8,101</b>',
                         textposition = "inside"))
fig.update_traces(hole=.4, hoverinfo="label+percent+name", pull=[0,0.1])
fig.update_layout(width=500, 
                  height=500, 
                  showlegend=True,
                  title_text="<b>유지/이탈 분포 현황<b>",
                  title_x = 0.5,
                  title_y = 0.9,
                  title_xanchor = "center",
                  title_yanchor = "middle")


fig.show()

#### age : 나이

In [ ]:
print_continuous_graphs(eda_churner_df, 'age', '나이')

- 나이에 대해 일부 이상치가 있지만, 대체적으로 유지 고객과 이탈 고객의 분포가 일치

In [ ]:
# 나이 범주화
eda_churner_df = category_func(eda_churner_df, 'age', 20, 10)
print_category_graphs(eda_churner_df, 'age_category', '나이 범주')

#### sex : 성별

In [ ]:
print_category_graphs(eda_churner_df, 'sex', '성별')

#### dependent_num : 부양가족수

In [ ]:
print_category_graphs(eda_churner_df, 'dependent_num', '부양가족수')

#### education : 교육수준
- Graduate : 대학원
- High School : 고졸
- Unknown
- Uneducated : 미교육
- College : 단과대학
- Post-Graduate : 보딩스쿨(재수)
- Doctorate :박사

In [ ]:
print_category_graphs(eda_churner_df, 'education', '교육수준')

- 시사점
    - 박사 학위를 받은 고객의 경우 이탈율이 더 높게 나타나고, 고등학교 및 대학 교육을 받은 고객은 이탈율이 더 낮은 것으로 보이나, 박사 학위 고객 수는 가장 낮음

    - 박사 학위의 경우 카드사의 혜택을 더 많이 고려하여 이동하는 것으로 보임   

#### marital_stat : 결혼상태
- Married  : 결혼
- Single   : 미혼
- Divorced : 이혼

In [ ]:
print_category_graphs(eda_churner_df, 'marital_stat', '결혼상태')

#### imcome_cat : 수입규모

In [ ]:
print_category_graphs(eda_churner_df, 'imcome_cat', '수입규모')

- 시사점
    - 소득 수입규모가 12만 달러 이상인 고객, 4만 달러 미만인 고객의 이탈율이 높게 나타나고 있으면,

    - 전체 고객대비 소득 수입규모가 40만 달러 미만인 고객이 차지하는 점유율이 높으므로 이 범주의 고객을 중점적으로 관리할 필요가 있음

    - 전반적으로 소득 수입규모별 감소율은 큰 차이가 없음

#### card_type : 카드종류

In [ ]:
print_category_graphs(eda_churner_df, 'card_type', '카드종류')

- 시사점
    - 플래티넘 카드 소유자의 건수는 매우 낮음에도 이탈율은 상당히 높게 나타나고 있음.

    - 이는 플래티넘 카드를 사용하는 고객의 만족도가 매우 낮은 것으로 보이며 플래티넘 카드의 혜택을 다른 카드사와 비교하여 부족한 부문을 찾아내어 개선하거나 고객이 만족할만한 혜택을 제공하여 이탈율을 낮출 필요가 있음 

    - 플래티넘 카드 소지자는 은행 서비스 이용을 중단하는 경향이 있습니다. 수수료가 너무 높거나 연간 서비스가 부족합니까? 수수료 결제됐나요?

#### mon_on_book : 은행 거래 기간 (개월 수)

In [ ]:
print_continuous_graphs(eda_churner_df, 'mon_on_book', '은행 거래 기간')

In [ ]:
eda_churner_df = category_func(eda_churner_df, 'mon_on_book', 10, 10)
print_category_graphs(eda_churner_df, 'mon_on_book_category', '은행거래기간 범주')

#### tot_product_count : 현재 보유 상품 개수

In [ ]:
print_category_graphs(eda_churner_df, 'tot_product_count', '현재 보유 상품 개수')

- 시사점
    - 이탈하지 않은 고객의 카드 보유 갯수의 중앙값은 4개이고 이탈한 고객의 중앙값은 3개임.
    - 카드의 개수가 적을수록 이탈율이 높아지므로 4개 이상의 카드를 보유할 수 있도록 노력 필요

#### months_inact_for_12m : 최근 12개월 동안 카드 거래가 없었던 개월 수

In [ ]:
print_category_graphs(eda_churner_df, 'months_inact_for_12m', '최근 12개월 동안 카드 거래가 없었던 개월 수')

- 시사점
    - 카드 거래 없는 개월수가 4개월 동안 없으면 이탈율이 정점을 찍고 이후 감소하는 경향을 보이고 있음
    
    - 이탈한 고객의 평균 기간은 3개월이고 유지하는 고객의 경우 2개월 임
    
    - 1개월 이상 카드 거래가 없는 고객은 잠재적으로 이탈할 확률이 높을 것으로 예상되므로 주기적으로 관리할 필요가 있음


#### contact_cnt_for_12m : 최근 12개월 동안 연락 횟수

In [ ]:
print_category_graphs(eda_churner_df, 'contact_cnt_for_12m', '최근 12개월 동안 연락 횟수')

- 시사점
   - 고객 접촉 건수와 이탈율 사이에는 명확한 관계가 있어 보임

   - 접촉 건수가 많을 수록 이탈율이 높아지는 경향을 보임

   - 데이터상 12개월동안 접촉건수가 고객이 접촉한 건수인지, 은행에서 접속한 건수 인지를 구분하여 분석할 필요가 있음
      - 은행이 고객을 접촉한 건수라면 연체 등으로 잦은 고객 독촉으로 이탈율이 높아질 가능성 있어 보이며 
      
      - 고객이 은행을 접촉한 건수라면 카드 관련 서비스의 불만을 원할하게 해결하지 못해 건수가 증가하고 이로인해 이탈율이 상승.<br>
   따라서, 카드관련 부서에서는 대고객 접촉 서비스를 세부적으로 분석할 필요가 있으며 대고객 대응 가이드를 점검해 볼 필요가 있음


   - 대고객 접촉 채널은 고객 행동을 분석하는데 있어 중요한 항목이므로 세부적으로 관리할 필요가 있으며 정기적으로 분석할 필요가 있음 
   - 접촉 채널, 접촉 주체, 접촉 내용 구분, 접촉 세부 내용, 문제해결 여부, 접촉에 대한 만족도 등 

#### credit_line : 카드 한도

In [ ]:
eda_churner_df = category_func(eda_churner_df, 'credit_line', 0, 5000)
print_category_graphs(eda_churner_df, 'credit_line_category', '카드 한도 범주')

#### tot_revol_balance : 리볼빙 잔액

In [ ]:
eda_churner_df = category_func(eda_churner_df, 'tot_revol_balance', 0, 500)
print_category_graphs(eda_churner_df, 'tot_revol_balance_category', '리볼빙 잔액 범주')

- 시사점
    - 신용카드 결제대상 잔액이 적을수록 이탈에 대한 비율이 커지므로 약 $1,000 이하 잔액이 있는 고객을 대상으로 마케팅할 필요가 있음  

#### mean_open_to_buy : 평균 사용가능 신용한도

In [ ]:
eda_churner_df = category_func(eda_churner_df, 'mean_open_to_buy', 0, 5000)
print_category_graphs(eda_churner_df, 'mean_open_to_buy_category', '평균 사용가능 신용한도 범주')

#### tot_amt_ratio_q4_q1 : 1분기 대비 4분기의 거래 금액 비율

In [ ]:
tot_cnt = eda_churner_df['tot_amt_ratio_q4_q1'].count().sum()
tot_null_cnt = eda_churner_df['tot_amt_ratio_q4_q1'].isnull().sum()
print(f'전체 데이터 건수 = {tot_cnt:,} Null 건수 = {tot_null_cnt:,} 전체 데이터 중 널 비율 =  {round(tot_null_cnt / tot_cnt,2)}') 

In [ ]:
eda_churner_df = category_func(eda_churner_df, 'tot_amt_ratio_q4_q1', 0, 0.2)
print_category_graphs(eda_churner_df, 'tot_amt_ratio_q4_q1_category', '1분기 대비 4분기의 거래 금액 비율 범주')

- 시사점
    - 1분기 대비 4분기의 거래 금액의 변화는 비슷하나 이탈하지 않은 고객의 분포를 보면 이상치가 많음. 

#### tot_trans_amt_for_12m : 최근 12개월 동안의 거래 금액

In [ ]:
tot_cnt = eda_churner_df['tot_trans_amt_for_12m'].count().sum()
tot_null_cnt = eda_churner_df['tot_trans_amt_for_12m'].isnull().sum()
print(f'전체 데이터 건수 = {tot_cnt:,} Null 건수 = {tot_null_cnt:,} 전체 데이터 중 널 비율 =  {round(tot_null_cnt / tot_cnt,2)}') 

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('최근 12개월 동안의 거래 금액 분포 및 이탈 현황')
sns.histplot(x='tot_trans_amt_for_12m', data=eda_churner_df, kde=True, hue='is_churned')

In [ ]:
eda_churner_df = category_func(eda_churner_df, 'tot_trans_amt_for_12m', 0, 2000)
print_category_graphs(eda_churner_df, 'tot_trans_amt_for_12m_category', '최근 12개월 동안의 거래 금액')

- 시사점
    - 이탈하지 않은 고객의 거래 금액이 더 높은 것으로 나타나고 있으나, 거래 금액별로 이탈율이 다르게 나타나고 있음

#### tot_trans_cnt_for_12m : 최근 12개월 동안의 거래 횟수

In [ ]:
tot_cnt = eda_churner_df['tot_trans_cnt_for_12m'].count().sum()
tot_null_cnt = eda_churner_df['tot_trans_cnt_for_12m'].isnull().sum()
print(f'전체 데이터 건수 = {tot_cnt:,} Null 건수 = {tot_null_cnt:,} 전체 데이터 중 널 비율 =  {round(tot_null_cnt / tot_cnt,2)}') 

In [ ]:
print_continuous_graphs(eda_churner_df, 'tot_trans_cnt_for_12m', '최근 12개월 동안의 거래 횟수')


In [ ]:
eda_churner_df = category_func(eda_churner_df, 'tot_trans_cnt_for_12m', 0, 20)
print_category_graphs(eda_churner_df, 'tot_trans_cnt_for_12m_category', '최근 12개월 동안의 거래 횟수')

- 시사점
    - 거래건수가 많을수록 유지되는 고객이 많으며 최근 12개월 동인 50번 이하의 고개 이탈율이 높아지므로 정기적으로 거래건수를 분석하여 대응할 필요가 있음

#### tot_cnt_ratio_q4_q1 : 1분기 대비 4분기의 거래 횟수 비율

In [ ]:
tot_cnt = eda_churner_df['tot_cnt_ratio_q4_q1'].count().sum()
tot_null_cnt = eda_churner_df['tot_cnt_ratio_q4_q1'].isnull().sum()
print(f'전체 데이터 건수 = {tot_cnt:,} Null 건수 = {tot_null_cnt:,} 전체 데이터 중 널 비율 =  {round(tot_null_cnt / tot_cnt,2)}') 

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('1분기 대비 4분기의 거래 횟수 분포 및 이탈 현황')
sns.histplot(x='tot_cnt_ratio_q4_q1', data=eda_churner_df, kde=True, hue='is_churned')

In [ ]:
eda_churner_df = category_func(eda_churner_df, 'tot_cnt_ratio_q4_q1', 0, 0.3)
print_category_graphs(eda_churner_df, 'tot_cnt_ratio_q4_q1_category', '1분기 대비 4분기의 거래 횟수 비율')

#### mean_util_pct : 평균 한도 소진율

In [ ]:
tot_cnt = eda_churner_df['mean_util_pct'].count().sum()
tot_null_cnt = eda_churner_df['mean_util_pct'].isnull().sum()
print(f'전체 데이터 건수 = {tot_cnt:,} Null 건수 = {tot_null_cnt:,} 전체 데이터 중 널 비율 =  {round(tot_null_cnt / tot_cnt,2)}') 

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('평균 한도 소진율 분포 및 이탈 현황')
sns.histplot(x='mean_util_pct', data=eda_churner_df, kde=True, hue='is_churned')

In [ ]:
eda_churner_df = category_func(eda_churner_df, 'mean_util_pct', 0, 0.2)
print_category_graphs(eda_churner_df, 'mean_util_pct_category', '평균 한도 소진율')

- 시사점
    - 한도 소진율이 낮을수록 이탈율은 올라가는 경향이 있음

#### 연속형 데이터 분포

In [ ]:
plt.figure(figsize=(18, 18))
# for i, col in enumerate(bank_churner_df.drop(['is_churned'], axis=1).select_dtypes(include=['int','float']).columns):
for i, col in enumerate(['age', 'mon_on_book', 'credit_line', 'tot_revol_balance','mean_open_to_buy','tot_amt_ratio_q4_q1','tot_trans_amt_for_12m','tot_trans_cnt_for_12m','mean_util_pct']):        
# for i, col in enumerate(['age', 'mon_on_book', 'credit_line' ]):    
    # We exclude the 'y' column and only consider the columns of numerical type.
    # Excluimos la columna 'y' y solo consideramos las columnas de tipo numérico.

    plt.rcParams['axes.facecolor'] = 'white'
    ax = plt.subplot(4, 4, i+1)  # Creating a subplot for each column.
    # Creamos una subfigura para cada columna.

     # Plotting the histogram for each column
    sns.histplot(data=eda_churner_df, x=col, ax=ax, color='red', kde=True)

    # Plotting the KDE curve with custom color and linewidth
    # Plotting the histogram for each column.
    # Graficamos el histograma para cada columna.
    ax.tick_params(axis='x', labelsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ax.set_xlabel(col, fontsize=18)
    ax.set_ylabel('Count', fontsize=18)
    
plt.suptitle('Data distribution of continuous variables',fontsize=18)
plt.tight_layout()

### Feature별 특징 분석 결과 요약

- 저소득층에 집중: 구매력은 크지 않지만 대부분의 고객은 저소득층입니다. 저소득층을 위한 프로모션을 시행하는 것은 해당 클러스터 그룹의 고객 이탈을 줄이는 좋은 대안이 될 수 있습니다.

- 활동 수준이 낮을 때 조치: 활동 수준이 낮은 고객(트랜잭션 45개 미만)이 조직을 떠날 확률이 더 높다는 것을 확인했습니다. 직원이 활동 수준이 낮은 고객에게 전화를 걸어 그들의 요구 사항에 맞는 새로운 제품을 제안하거나 고객이 우리가 제공하는 서비스에 만족하는지, 개선하기 위해 할 수 있는 것이 있는지 묻는다면 우리는 아마도 무엇에 대해 더 나은 통찰력을 얻을 수 있을 것입니다. 우리는 활동 수준을 높이기 위해 할 수 있습니다.

- 회전 잔액이 적은 사람들에게 신용 한도를 늘리시겠습니까? 우리 모델에 따르면 회전 잔액이 낮은 고객은 조직을 떠날 가능성이 더 높습니다. 어쩌면 해당 고객에게 더 높은 신용 잔액을 구현함으로써 해당 세그먼트 그룹이 조직을 떠날 확률이 낮아질 수 있습니다.


- 기술적 분석
    - 거래금액과 거래횟수가 많을수록 이탈 가능성이 낮아짐
    - 플리티넘 카드 보유자 수가 적고 회원 탈퇴율이 높아 좀 더 관심을 가져야 할 분야
    - 이탈율은 비활성화 된지 4개월이 지나면 최고조에 달하니 이러한 현생이 발생한 이유를 좀 더 세부적으로 분석할 필요가 있음

    - 카드의 종류가 많을수록 이탈 가능성이 줄어들며
    - 접촉 건수에 따른 이탈율을 좀더 분석할 필요가 있음

### 다변량 분석

In [ ]:
eda_churner_df.info()

In [ ]:
sns.histplot(data=eda_churner_df, x='sex', y='age', hue='is_churned')

In [ ]:
sns.barplot(data=eda_churner_df, x='sex', y='age', hue='is_churned')

#### 성별, 카드 종류별 비율

In [ ]:
fig = make_subplots(
    rows=2, cols=2,subplot_titles=('','<b>Platinum Card Holders','<b>Blue Card Holders<b>','Residuals'),
    vertical_spacing=0.09,
    specs=[[{"type": "pie","rowspan": 2}       ,{"type": "pie"}] ,
           [None                               ,{"type": "pie"}]            ,                                      
          ]
)

fig.add_trace(
    go.Pie(values=bank_churner_df.sex.value_counts().values,labels=['<b>여자<b>','<b>남자<b>'],hole=0.3,pull=[0,0.3]),
    row=1, col=1
)

fig.add_trace(
    go.Pie(
        labels=['Female Platinum Card Holders','Male Platinum Card Holders'],
        values=bank_churner_df.query('card_type=="Platinum"').sex.value_counts().values,
        pull=[0,0.05,0.5],
        hole=0.3
        
    ),
    row=1, col=2
)

# fig.add_trace(
#     go.Pie(
#         labels=['Female Gold Card Holders','Male Blue Card Holders'],
#         values=bank_churner_df.query('card_type=="Gold"').sex.value_counts().values,
#         pull=[0,0.2,0.5],
#         hole=0.3
#     ),
#     row=2, col=1
# )

fig.add_trace(
    go.Pie(
        labels=['Female Silver Card Holders','Male Blue Card Holders'],
        values=bank_churner_df.query('card_type=="Silver"').sex.value_counts().values,
        pull=[0,0.2,0.5],
        hole=0.3
    ),
    row=2, col=2
)



fig.update_layout(
    height=800,
    showlegend=True,
    title_text="<b>Distribution Of Gender And Different Card Statuses<b>",
)

fig.show()

## Machine Learning
***

### Import Library and Data Loading, Function Definition for Machine Learning

#### Import Library

In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve

# import scikitplot as skplt
from imblearn.over_sampling import SMOTE

#### Function Definition

In [ ]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test, pred)
    f11 = f1_score(y_test, pred, average='weighted')
    roc_auc = roc_auc_score(y_test, pred_proba)

    # print('오차 행렬')
    # print(confusion)
    
    # ROC-AUC print 추가
    # ------------------
    # print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    # F1: {3:.4f}, F11: {4:.4f}, AUC:{5:.4f}'.format(accuracy, precision, recall, f1, f11, roc_auc))   
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
          F1: {3:.4f}, F11: {4:.4f}, AUC:{5:.4f}'.format(accuracy, precision, recall, f1, f11, roc_auc))   

In [ ]:
def precision_recall_curve_plot(y_test=None, pred_proba=None):
    # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출. 
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba)
    
    # X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 Plot 수행. 정밀도는 점선으로 표시
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary],label='recall')
    
    # threshold 값 X 축의 Scale을 0.1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    
    # x축, y축 label과 legend, 그리고 grid 설정
    plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall value')
    plt.legend(); plt.grid()
    plt.show()

In [ ]:
def precision_recall_curve_plot(y_test=None, pred_proba=None):
    # threshold ndarray와 이 threshold에 따른 정밀도, 재현율 ndarray 추출. 
    precisions, recalls, thresholds = precision_recall_curve(y_test, pred_proba)
    
    # X축을 threshold값으로, Y축은 정밀도, 재현율 값으로 각각 Plot 수행. 정밀도는 점선으로 표시
    plt.figure(figsize=(8,6))
    threshold_boundary = thresholds.shape[0]
    plt.plot(thresholds, precisions[0:threshold_boundary], linestyle='--', label='precision')
    plt.plot(thresholds, recalls[0:threshold_boundary],label='recall')
    
    # threshold 값 X 축의 Scale을 0.1 단위로 변경
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    
    # x축, y축 label과 legend, 그리고 grid 설정
    plt.xlabel('Threshold value'); plt.ylabel('Precision and Recall value')
    plt.legend(); plt.grid()
    plt.show()

In [ ]:
def roc_curve_plot(y_test , pred_proba):
    # 임곗값에 따른 FPR, TPR 값을 반환 받음. 
    fprs , tprs , thresholds = roc_curve(y_test ,pred_proba)

    # ROC Curve를 plot 곡선으로 그림. 
    plt.plot(fprs , tprs, label='ROC')
    # 가운데 대각선 직선을 그림. 
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    
    # FPR X 축의 Scale을 0.1 단위로 변경, X,Y 축명 설정등   
    start, end = plt.xlim()
    plt.xticks(np.round(np.arange(start, end, 0.1),2))
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel('FPR( 1 - Sensitivity )'); plt.ylabel('TPR( Recall )')
    plt.legend()
    plt.show()
    

#### Data Loading

In [ ]:
bank_churner_df = pd.read_csv("./data/bank_churner.csv")

### 전처리(Pre-Processing)

#### 유사도, 상관도 분석 - 다중공선성 확인

In [ ]:
sns.set(rc={'figure.figsize':(20,20)}) # 그래프 크기

corr = bank_churner_df.corr() # 상관행렬 표 만들기
sns.heatmap(round(corr,1), 
            annot=True, # 상관계수 표시
            fmt='.1f', # 상관계수 소수점 자리
            cmap='coolwarm', # 컬러맵 색상 팔레트 
            vmax=1.0, # 상관계수 최댓값 
            vmin=-1.0, # 상관계수 최솟값
            linecolor='white', # 셀 테두리 색상 
            linewidths=.05) # 셀 간격 

In [ ]:
sns.pairplot(hue = 'is_churned', data = bank_churner_df)
plt.show()

이 그래프를 시각적으로 분석하면 특정 변수에 대한 명확한 클러스터링 패턴을 관찰할 수 있습니다. 패턴이 다음과 같을 때 관심이 갑니다.
신용 한도의 대각선 그래프를 보면 은행을 떠나기로 결정한 사람들은 신용 한도가 낮은 사람들입니다.
신용한도가 높은 사람들은 은행을 떠나지 않기로 결정합니다(파란색 그래프의 정점). 신용 한도와 다른 변수를 그래프로 표시할 때,
흥미로운 클러스터링 패턴을 찾을 수 있습니다.
얼핏 보면 고객의 체류 여부를 결정하는 데 다음과 같은 변수가 중요한 영향을 미치는 것으로 보입니다.

age, credit_lile, tot_revol_balance, tot_amt_ratio_q4_q1, tot_trans_amt_for_12m, tot_cnt_ratio_q4_q1, mean_util_pct

- 시사점
    - 다중공선성: 다중공선성은 회귀 모델에서 두 개 이상의 독립변수가 높은 상관관계를 가질 때 발생합니다. 이로 인해 가변 계수의 해석이 어려워지고 모델의 안정성과 신뢰성이 낮아질 수 있습니다.

    - age와 mon_on_book, credit_line과 mean_open_to_buy, tot_trans_cnt_for_12m와 tot_trans_amt_for_12m 사이에도 강한 상관관계가 있음을 관찰
이로 인해 mon_on_book, mean_open_to_buy, tot_trans_cnt_for_12m 열을 제거할 예정입니다.

컬럼명               Null 건수
------------------- ---------  
age                         0 - 삭제1
sex                       808
imcome_cat               1619
mon_on_book                 0 - 삭제1 : 삭제
credit_line                 0 - 삭제2
tot_revol_balance        1521                 - 공선성1 
mean_open_to_buy            0 - 삭제2 : 삭제 
tot_amt_ratio_q4_q1      2435
tot_trans_amt_for_12m    1669 - 삭제3
tot_trans_cnt_for_12m    3250 - 삭제3 : 삭제
tot_cnt_ratio_q4_q1      1629
mean_util_pct            2526                 - 공선성1 

In [ ]:
bank_churner_df = bank_churner_df[['cstno', 'is_churned', 'age', 'dependent_num', 'tot_product_count', 'months_inact_for_12m',
        'contact_cnt_for_12m', 'credit_line', 'tot_revol_balance',
        'tot_amt_ratio_q4_q1', 'tot_trans_amt_for_12m', 'tot_cnt_ratio_q4_q1']]
bank_churner_df.info()

In [ ]:
sns.set(rc={'figure.figsize':(20,20)}) # 그래프 크기
corr = bank_churner_df.corr() # 상관행렬 표 만들기
sns.heatmap(round(corr,1), 
            annot=True, # 상관계수 표시
            fmt='.1f', # 상관계수 소수점 자리
            cmap='coolwarm', # 컬러맵 색상 팔레트 
            vmax=1.0, # 상관계수 최댓값 
            vmin=-1.0, # 상관계수 최솟값
            linecolor='white', # 셀 테두리 색상 
            linewidths=.05) # 셀 간격 


In [ ]:
sns.pairplot(hue = 'is_churned', data = bank_churner_df)
plt.show()

#### Null 확인

#### 전처리 함수 정의

In [ ]:
def test_transform(x_test):
    ''' 전처리 함수 정의'''
    
    # 불필요 컬럼 제거(고객번호)
    # -------------------------
    x_test = x_test.drop('cstno', axis=1)
    
    
    # 성별 변환('F':0, 'M':1)
    # -------------------------
    # x_test['sex']=x_test['sex'].replace({'F':0,'M':1})
    
    
    # 다중공선성 컬럼 제거
    # -------------------
    # x_test = x_test.drop('mon_on_book', axis = 1)
    # x_test = x_test.drop('mean_open_to_buy', axis = 1)
    # x_test = x_test.drop('tot_trans_cnt_for_12m', axis = 1)

    # x_test = x_test[['is_churned', 'age', 'dependent_num', 'tot_product_count', 'months_inact_for_12m',
    #     'contact_cnt_for_12m', 'credit_line', 'tot_revol_balance',
    #     'tot_amt_ratio_q4_q1', 'tot_trans_amt_for_12m', 'tot_cnt_ratio_q4_q1']]

    # 범주형 데이터 One-Hot 인코딩
    # --------------------------
    # x_test = pd.concat([x_test,pd.get_dummies(x_test['education']).drop(columns=['Unknown'])],axis=1)
    # x_test = pd.concat([x_test,pd.get_dummies(x_test['imcome_cat']).drop(columns=['Unknown'])],axis=1)
    # x_test = pd.concat([x_test,pd.get_dummies(x_test['marital_stat']).drop(columns=['Unknown'])],axis=1)
    # x_test = pd.concat([x_test,pd.get_dummies(x_test['card_type']).drop(columns=['Platinum'])],axis=1)
    # x_test.drop(columns = ['education','imcome_cat','marital_stat','card_type'],inplace=True)


    # Null 처리 1 방식
    # ---------------
    # x_test.dropna(axis=0, inplace=True)

    
    # Null 처리 2 방식
    # ---------------
    # x_test.drop(columns = ['sex'], inplace=True)
    # x_test.drop(columns = ['tot_revol_balance'], inplace=True)
    # x_test.drop(columns = ['tot_amt_ratio_q4_q1'], inplace=True)        
    # x_test.drop(columns = ['tot_trans_amt_for_12m'], inplace=True)        
    # x_test.drop(columns = ['tot_cnt_ratio_q4_q1'], inplace=True)        
    # x_test.drop(columns = ['mean_util_pct'], inplace=True)


    # # Null 처리 3 방식
    # # ----------------
 #   x_test.drop(columns = ['mean_util_pct'], inplace=True)
    x_test.dropna(axis=0, inplace=True)
        
    return x_test

#### 데이터 전처리 수행

In [ ]:
bank_churner_df = test_transform(bank_churner_df)
bank_churner_df.shape

In [ ]:
bank_churner_df.info()

In [ ]:
#We create our feature matrix and our target variable vector.
X=bank_churner_df.drop(['is_churned'],axis=1)
y=bank_churner_df['is_churned']

#### 학습에 사용될 중요 Feature 식별

In [ ]:
'''
#Selection of the most important features to conduct the training
#Selección de las características más importantes para llevar a cabo el entrenamiento
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.feature_selection import RFE
import pandas as pd


# Establecer la semilla aleatoria para reproducibilidad
#Set the random seed for reproducibility
np.random.seed(42)

#Define a list of available models for selection
# Definir una lista de modelos disponibles para selección
available_models = {
    'ExtraTrees': ExtraTreesClassifier(n_estimators=100),
    'RandomForest': RandomForestClassifier(n_estimators=100),
    #'SVM': SVC(kernel='linear'),
    #'KNN': KNeighborsClassifier(n_neighbors=5),
    #'LASSO': Lasso(alpha=0.01),  # Agrega LASSO aquí
    #'RFE': RFE(estimator=RandomForestClassifier(n_estimators=100), n_features_to_select=10)
    # Agrega otros modelos aquí si lo deseas
}

# Choose the desired model for feature selection
chosen_model = 'ExtraTrees'

# Create the selected model
clf = available_models[chosen_model]

#Train the model with the data
clf = clf.fit(X.values, y)

# Obtain feature importances from the model
feature_importances = clf.feature_importances_

# Create a SelectFromModel object with the trained classifier
model = SelectFromModel(clf, prefit=True)

#Transform the original features to obtain the selected ones
X_new = model.transform(X.values)

selected_feature_indices = model.get_support(indices=True)

#Get the indices of the selected features
selected_columns = X.columns[selected_feature_indices]

#Print the selected columns
print("Selected columns:")
print(selected_columns)

'''

In [ ]:
'''
#Based on the analysis of the graphs, we had predicted that:
#At first glance, the following variables seem to have a significant influence on the determination of whether customers stay or not: Customer_Age, Credit_Limit,
#Total_Recovering_Bal, Total_Amt_Chng_Q4_Q1, Total_Trans_Amt, Total_Ct_Chng_Q4_Q1, Avg_Utilization_Ratio
#It seems that our intuition was correct.

import matplotlib.pyplot as plt
import seaborn as sns

#Get the indices of all columns in descending order of importance
sorted_indices = feature_importances.argsort()[::-1]

#Get the names of all columns in the same order
sorted_columns = X.columns[sorted_indices]

#Get the sorted importances
sorted_importances = feature_importances[sorted_indices]

plt.figure(figsize=(10, 6))

#Create a bar chart to display the importance of all columns in descending order
sns.barplot(x=sorted_importances, y=sorted_columns, palette=['lightgrey' if i not in selected_feature_indices else 'blue' for i in sorted_indices])

plt.xlabel("Importance", fontsize=14)
plt.ylabel("Feattures", fontsize=14)
plt.title("Feature Importance", fontsize=16)
plt.yticks(rotation=0, fontsize=12)
plt.show()

'''

In [ ]:
X_new = X

In [ ]:
X_new.shape

In [ ]:
#Model Training
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
X_train,X_test,y_train,y_test=train_test_split(X_new,y,test_size=0.25,stratify=y,random_state=0)

#stratify=y: It is used to ensure that the distribution of classes in the training and test sets is similar to the original distribution 
#of the target variable y. This is particularly useful when dealing with umbalanced classes, as it ensures that both parts of the split have
#a similar proportion of each class.

In [ ]:
#Verifying the size of the training and testing sets
print("Training X size: ", X_train.shape)
print("Training y size: ", y_train.shape)
print("Test X size: ", X_test.shape)
print("Test y size: ", y_test.shape)

In [ ]:
import matplotlib.pyplot as plt

plt.bar(['No', 'Yes'], y_train.value_counts(), color=['blue', 'orange'])
plt.xlabel('Response', fontsize=18)
plt.ylabel('Couts', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.title('Target Variable Distribution', fontsize=16)

plt.show()

In [ ]:
#SMOTE(Synthetic Minority Oversampling Technique)
#SMOTE is a technique for oversampling the minority class. Simply adding duplicate records of the minority class often does not add new
#information to the model. In SMOTE, new instances are generated from the existing data. To put it simply, SMOTE examines instances of 
#the minority class and uses the k-nearest neighbors method to select a randomly close neighbor, and a new synthetic instance is created 
#in the feature space.

from imblearn.over_sampling import SMOTE
X_train,X_test,y_train,y_test=train_test_split(X_new,y,test_size=0.25,stratify=y,random_state=0)

sm = SMOTE(sampling_strategy='auto', random_state=42)
X_train,y_train=sm.fit_resample(X_train,y_train)

In [ ]:
X_train.shape

In [ ]:
import matplotlib.pyplot as plt

plt.bar(['No', 'Yes'], y_train.value_counts(), color=['blue', 'orange'])
plt.xlabel('Response', fontsize=18)
plt.ylabel('Couts', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.title('Target Variable Distribution', fontsize=16)

plt.show()

#### Normalization

In [ ]:
#Normalization
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

### 모델별 학습 및 평가

In [ ]:
#Training with different models
#entrenamiento con distintos modelos
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#Create a list of tuples with the model name and the classifier instance
# Crear una lista de tuplas con el nombre del modelo y la instancia del clasificador
models = [
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier(criterion='entropy', random_state=0)),
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    ('Naive Bayes', GaussianNB()),
    ('Random Forest', RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)),
    ('Xg Boost', XGBClassifier())
]

model_comparison = {}  #Dictionary to store the comparison metrics of models
                        # Diccionario para almacenar las métricas de comparación de modelos

for model_name, classifier in models:
    #Fit the model using the training set
    classifier.fit(X_train, y_train)


    #Make predictions on the test set
    y_pred = classifier.predict(X_test)
    pred_proba = classifier.predict_proba(X_test)[:, 1]

    
    #Calculate model metrics
    accuracy = accuracy_score(y_test, y_pred)
    #f1 = f1_score(y_pred, y_test, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    ## 확인 필요 ??? 
    accuracies = cross_val_score(estimator=classifier, X=X_test, y=y_test, cv=5, scoring="recall")
    cv_accuracy = accuracies.mean()
    cv_std = accuracies.std()
    accuracy_class_0 = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0])
    accuracy_class_1 = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1], )
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    #Print model metrics
    print("-" * 30)
    print(f"Model: {model_name}")
    print("-" * 30)
    # print(f"Model Accuracy: {accuracy * 100:.2f}%")
    # print(f"Model F1-Score: {f1 * 100:.2f}%")
    # print(f"Cross Val Accuracy: {cv_accuracy * 100:.2f}%")
    # print(f"Cross Val Standard Deviation: {cv_std * 100:.2f}%")


    #Add metrics to the models comparison dictionary
    model_comparison[model_name] = [accuracy, accuracy_class_0, accuracy_class_1, f1, cv_accuracy, cv_std, roc_auc]
    # print(classification_report(y_pred, y_test, zero_division=1))
    # print("-" * 60)

    
    get_clf_eval(y_test, y_pred, pred_proba)
    # precision_recall_curve_plot(y_test, pred_proba)
    # roc_curve_plot(y_test , pred_proba)
    
    print("-" * 100)



### VotingClassifier

In [ ]:
#Ensemble methods in machine learning involve combining multiple models (often weaker models or base models) to create a stronger,
#more robust predictive model. The idea behind ensembling is that by combining the predictions of multiple models, the strengths 
#of each individual model can compensate for the weaknesses of others, leading to improved overall performance.
from sklearn.ensemble import VotingClassifier

models = [
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier(criterion='entropy', random_state=0)),
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    ('Naive Bayes', GaussianNB()),
    ('Random Forest', RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)),
    ('Xg Boost', XGBClassifier())
]

voting_classifier = VotingClassifier(estimators=models, voting='soft')  # Puedes usar 'hard' o 'soft' para el voto

voting_classifier.fit(X_train, y_train)

y_pred = voting_classifier.predict(X_test)
pred_proba = classifier.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
accuracies = cross_val_score(estimator=voting_classifier, X=X_test, y=y_test, cv=5, scoring="recall")
cv_accuracy = accuracies.mean()
cv_std = accuracies.std()
accuracy_class_0 = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0] )
accuracy_class_1 = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1])

roc_auc = roc_auc_score(y_test, pred_proba)
    

print("Model: Voting Classifier")
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print(f"Model F1-Score: {f1 * 100:.2f}%")
print(f"Cross Val Accuracy: {cv_accuracy * 100:.2f}%")
print(f"Cross Val Standard Deviation: {cv_std * 100:.2f}%")

model_comparison['Voting Classifier'] = [accuracy, accuracy_class_0, accuracy_class_1, f1, cv_accuracy, cv_std, roc_auc]
print(classification_report(y_test, y_pred, zero_division=1))
print("-" * 60)

In [ ]:
# MODEL COMPARISSON

Model_com_df=pd.DataFrame(model_comparison).T
Model_com_df.columns=['Model Accuracy','Model Accuracy-0','Model Accuracy-1','Model F1-Score','CV Accuracy','CV std', 'AUC']
Model_com_df=Model_com_df.sort_values(by='AUC',ascending=False)
Model_com_df.style.format("{:.2%}").background_gradient(cmap='magma')

In [ ]:
import pandas as pd

Model_com_df = pd.DataFrame(model_comparison).T
Model_com_df.columns = ['Model Accuracy', 'Model Accuracy-No', 'Model Accuracy-Yes', 'Model F1-Score', 'CV Accuracy', 'CV std', 'AUC']
Model_com_df = Model_com_df.sort_values(by='AUC', ascending=False)

def highlight_below_75(s):
    if s.name != 'CV std' and isinstance(s, pd.Series) and s.dtype == 'float64':
        return ['color: red' if value < 0.75 else 'color: black' for value in s]
    else:
        return ['color: black'] * len(s)

styled_df = Model_com_df.style.highlight_max(axis=0).apply(highlight_below_75, subset=pd.IndexSlice[:, :'CV Accuracy']).format("{:.2%}", subset=pd.IndexSlice[:, :'CV Accuracy'])
styled_df

## Deep Learning
***

In [ ]:
#!pip install tensorflow

### Import Library and Data Loading, Function Definition for Machine Learning

#### Import Library

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report

from sklearn.metrics import auc

#### Data Loading

In [ ]:
bank_churner_df = pd.read_csv("./data/bank_churner.csv")

### 전처리(Pre-Processing)

In [ ]:
# 테스트 데이터 전처리

def test_transform(x_test):
    ''' 전처리 함수 정의'''
    
    # 불필요 컬럼 제거(고객번호)
    # -------------------------
    x_test = x_test.drop('cstno', axis=1)
    
    
    # 성별 변환('F':0, 'M':1)
    # -------------------------
    x_test['sex']=x_test['sex'].replace({'F':0,'M':1})
    x_test['is_churned']=x_test['is_churned'].replace({'Existing Customer':0,'Attrited Customer':1})
    
    
    # 다중공선성 컬럼 제거
    # -------------------
    x_test = x_test.drop('mon_on_book', axis = 1)
    x_test = x_test.drop('mean_open_to_buy', axis = 1)
    x_test = x_test.drop('tot_trans_cnt_for_12m', axis = 1)


    # 범주형 데이터 One-Hot 인코딩
    # --------------------------
    x_test = pd.concat([x_test,pd.get_dummies(x_test['education']).drop(columns=['Unknown'])],axis=1)
    x_test = pd.concat([x_test,pd.get_dummies(x_test['imcome_cat']).drop(columns=['Unknown'])],axis=1)
    x_test = pd.concat([x_test,pd.get_dummies(x_test['marital_stat']).drop(columns=['Unknown'])],axis=1)
    x_test = pd.concat([x_test,pd.get_dummies(x_test['card_type']).drop(columns=['Platinum'])],axis=1)
    x_test.drop(columns = ['education','imcome_cat','marital_stat','card_type'],inplace=True)


    # Null 처리 1 방식
    # ---------------
    # x_test.dropna(axis=0, inplace=True)

    
    # Null 처리 2 방식
    # ---------------
    # x_test.drop(columns = ['sex'], inplace=True)
    # x_test.drop(columns = ['tot_revol_balance'], inplace=True)
    # x_test.drop(columns = ['tot_amt_ratio_q4_q1'], inplace=True)        
    # x_test.drop(columns = ['tot_trans_amt_for_12m'], inplace=True)        
    # x_test.drop(columns = ['tot_cnt_ratio_q4_q1'], inplace=True)        
    # x_test.drop(columns = ['mean_util_pct'], inplace=True)


    # # Null 처리 3 방식
    # # ----------------
    x_test.drop(columns = ['mean_util_pct'], inplace=True)
    x_test.dropna(axis=0, inplace=True)
        
    return x_test

In [ ]:
bank_churner_df = test_transform(bank_churner_df)

y=bank_churner_df['is_churned']
X_new=bank_churner_df.drop(['is_churned'], axis=1)

X_new = X_new[['age', 'dependent_num', 'tot_product_count', 'months_inact_for_12m',
       'contact_cnt_for_12m', 'credit_line', 'tot_revol_balance',
       'tot_amt_ratio_q4_q1', 'tot_trans_amt_for_12m', 'tot_cnt_ratio_q4_q1']]

X_train,X_test,y_train,y_test=train_test_split(X_new,y,test_size=0.25,stratify=y,random_state=0)

#Normalization
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [ ]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets
X_train,X_test,y_train,y_test=train_test_split(X_new,y,test_size=0.25,stratify=y,random_state=0)

In [ ]:
#DEEP LEARNING

# Scale the data with StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

# Build the model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Predict on the test data
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba >= 0.5).astype(int)


# --------------------------------
y_pred = model.predict(X_test).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred)

from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)
print('auc_keras: ')
# --------------------------------

# Calculate metrics
# print(f"Model Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
# print(f"Model F1-Score: {f1_score(y_test, y_pred, average='weighted') * 100:.2f}%")
# print(classification_report(y_test, y_pred, zero_division=1))

# Calculate accuracies per class
# accuracy_class_0 = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0])
# accuracy_class_1 = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1])

#pred_proba_proba = model.predict_proba(X_test)[:, 1]
from sklearn.ensemble import RandomForestClassifier
# Supervised transformation based on random forests
rf = RandomForestClassifier(max_depth=3, n_estimators=10)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_pred_rf)
auc_rf = auc(fpr_rf, tpr_rf)

#get_clf_eval(y_test, y_pred, y_pred_proba)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')
plt.show()


In [ ]:
auc_keras

In [ ]:
'''
# Compute confusion matrix for the Deep learning model
from sklearn.metrics import classification_report, confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Reds):
    """
    This function prints and plots the confusion matrix. Normalization can be applied by setting normalize=True.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix without normalization')

    print(cm)

    #Plot the confusion matrix.
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.grid(False)  # <-- Agregar esta línea para evitar el aviso de deprecación
    plt.title(title,fontsize=18)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    # Add labels to the cells of the confusion matrix.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",fontsize=16,
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label',fontsize=15)
    plt.xlabel('Predicted label',fontsize=15)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

print(confusion_matrix(y_test, y_pred, labels=[0,1]))


cnf_matrix = confusion_matrix(y_test, y_pred, labels=[0,1])
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['y=0','y=1'],normalize= True,  title='Confusion matrix')

'''

## 최종 테스트 데이터로 평가
***

In [ ]:
test_df = pd.read_csv("./data/test_churner.csv")
test_df.info()

In [ ]:
# 테스트 데이터 전처리

def test_transform(x_test):
    ''' 전처리 함수 정의'''
    
    # 불필요 컬럼 제거(고객번호)
    # -------------------------
    x_test = x_test.drop('cstno', axis=1)
    
    
    # 성별 변환('F':0, 'M':1)
    # -------------------------
    x_test['sex']=x_test['sex'].replace({'F':0,'M':1})
    x_test['is_churned']=x_test['is_churned'].replace({'Existing Customer':0,'Attrited Customer':1})
    
    x_test = x_test[['is_churned', 'age', 'dependent_num', 'tot_product_count', 'months_inact_for_12m',
        'contact_cnt_for_12m', 'credit_line', 'tot_revol_balance',
        'tot_amt_ratio_q4_q1', 'tot_trans_amt_for_12m', 'tot_cnt_ratio_q4_q1']]
    
    # 다중공선성 컬럼 제거
    # -------------------
    # x_test = x_test.drop('mon_on_book', axis = 1)
    # x_test = x_test.drop('mean_open_to_buy', axis = 1)
    # x_test = x_test.drop('tot_trans_cnt_for_12m', axis = 1)


    # 범주형 데이터 One-Hot 인코딩
    # --------------------------
    # x_test = pd.concat([x_test,pd.get_dummies(x_test['education']).drop(columns=['Unknown'])],axis=1)
    # x_test = pd.concat([x_test,pd.get_dummies(x_test['imcome_cat']).drop(columns=['Unknown'])],axis=1)
    # x_test = pd.concat([x_test,pd.get_dummies(x_test['marital_stat']).drop(columns=['Unknown'])],axis=1)
    # x_test = pd.concat([x_test,pd.get_dummies(x_test['card_type']).drop(columns=['Platinum'])],axis=1)
    # x_test.drop(columns = ['education','imcome_cat','marital_stat','card_type'],inplace=True)


    # Null 처리 1 방식
    # ---------------
    # x_test.dropna(axis=0, inplace=True)

    
    # Null 처리 2 방식
    # ---------------
    # x_test.drop(columns = ['sex'], inplace=True)
    # x_test.drop(columns = ['tot_revol_balance'], inplace=True)
    # x_test.drop(columns = ['tot_amt_ratio_q4_q1'], inplace=True)        
    # x_test.drop(columns = ['tot_trans_amt_for_12m'], inplace=True)        
    # x_test.drop(columns = ['tot_cnt_ratio_q4_q1'], inplace=True)        
    # x_test.drop(columns = ['mean_util_pct'], inplace=True)


    # # Null 처리 3 방식
    # # ----------------
    # x_test.drop(columns = ['mean_util_pct'], inplace=True)
    x_test.dropna(axis=0, inplace=True)
        
    return x_test



In [ ]:
test_df = test_transform(test_df)

y_test=test_df['is_churned']
X_test=test_df.drop(['is_churned'], axis=1)

X_test=sc.transform(X_test)

In [ ]:
len(X_test)

In [ ]:
#Training with different models
#entrenamiento con distintos modelos
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
#Create a list of tuples with the model name and the classifier instance
# Crear una lista de tuplas con el nombre del modelo y la instancia del clasificador
models = [
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier(criterion='entropy', random_state=0)),
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    ('Naive Bayes', GaussianNB()),
    ('Random Forest', RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)),
    ('Xg Boost', XGBClassifier())
]


model_eval_comparison = {}  #Dictionary to store the comparison metrics of models

for model_name, classifier in models:
    #Fit the model using the training set
    classifier.fit(X_train, y_train)


    #Make predictions on the test set
    y_pred = classifier.predict(X_test)
    pred_proba = classifier.predict_proba(X_test)[:, 1]

    
    #Calculate model metrics
    accuracy = accuracy_score(y_test, y_pred)
    #f1 = f1_score(y_pred, y_test, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracies = cross_val_score(estimator=classifier, X=X_test, y=y_test, cv=5, scoring="recall")
    cv_accuracy = accuracies.mean()
    cv_std = accuracies.std()
    accuracy_class_0 = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0])
    accuracy_class_1 = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1])
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    
    #Print model metrics
    print("-" * 30)
    print(f"Model: {model_name}")
    print("-" * 30)
    # print(f"Model Accuracy: {accuracy * 100:.2f}%")
    # print(f"Model F1-Score: {f1 * 100:.2f}%")
    # print(f"Cross Val Accuracy: {cv_accuracy * 100:.2f}%")
    # print(f"Cross Val Standard Deviation: {cv_std * 100:.2f}%")


    # #Add metrics to the models comparison dictionary
    model_eval_comparison[model_name] = [accuracy, accuracy_class_0, accuracy_class_1, f1, cv_accuracy, cv_std, roc_auc]
    # print(classification_report(y_pred, y_test, zero_division=1))
    # print("-" * 60)

    
    get_clf_eval(y_test, y_pred, pred_proba)
    # precision_recall_curve_plot(y_test, pred_proba)
    # roc_curve_plot(y_test , pred_proba)
    
    print("-" * 100)



### VotingClassifier

In [ ]:
#Ensemble methods in machine learning involve combining multiple models (often weaker models or base models) to create a stronger,
#more robust predictive model. The idea behind ensembling is that by combining the predictions of multiple models, the strengths 
#of each individual model can compensate for the weaknesses of others, leading to improved overall performance.
from sklearn.ensemble import VotingClassifier

models = [
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier(criterion='entropy', random_state=0)),
    ('KNN', KNeighborsClassifier(n_neighbors=5)),
    ('Naive Bayes', GaussianNB()),
    ('Random Forest', RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)),
    ('Xg Boost', XGBClassifier())
]

voting_classifier = VotingClassifier(estimators=models, voting='soft')  # Puedes usar 'hard' o 'soft' para el voto

voting_classifier.fit(X_train, y_train)

y_pred = voting_classifier.predict(X_test)
pred_proba = classifier.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
accuracies = cross_val_score(estimator=voting_classifier, X=X_test, y=y_test, cv=5, scoring="recall")
cv_accuracy = accuracies.mean()
cv_std = accuracies.std()
accuracy_class_0 = accuracy_score(y_test[y_test == 0], y_pred[y_test == 0] )
accuracy_class_1 = accuracy_score(y_test[y_test == 1], y_pred[y_test == 1])

roc_auc = roc_auc_score(y_test, pred_proba)
    

print("Model: Voting Classifier")
print(f"Model Accuracy: {accuracy * 100:.2f}%")
print(f"Model F1-Score: {f1 * 100:.2f}%")
print(f"Cross Val Accuracy: {cv_accuracy * 100:.2f}%")
print(f"Cross Val Standard Deviation: {cv_std * 100:.2f}%")

model_eval_comparison['Voting Classifier'] = [accuracy, accuracy_class_0, accuracy_class_1, f1, cv_accuracy, cv_std, roc_auc]
print(classification_report(y_test, y_pred, zero_division=1))
print("-" * 60)

In [ ]:
# MODEL COMPARISSON

Model_com_df=pd.DataFrame(model_eval_comparison).T
Model_com_df.columns=['Model Accuracy','Model Accuracy-0','Model Accuracy-1','Model F1-Score','CV Accuracy','CV std', 'AUC']
Model_com_df=Model_com_df.sort_values(by='AUC',ascending=False)
Model_com_df.style.format("{:.2%}").background_gradient(cmap='magma')

In [ ]:
import pandas as pd

Model_com_df = pd.DataFrame(model_eval_comparison).T
Model_com_df.columns = ['Model Accuracy', 'Model Accuracy-No', 'Model Accuracy-Yes', 'Model F1-Score', 'CV Accuracy', 'CV std', 'AUC']
Model_com_df = Model_com_df.sort_values(by='AUC', ascending=False)

def highlight_below_75(s):
    if s.name != 'CV std' and isinstance(s, pd.Series) and s.dtype == 'float64':
        return ['color: red' if value < 0.75 else 'color: black' for value in s]
    else:
        return ['color: black'] * len(s)

styled_df = Model_com_df.style.highlight_max(axis=0).apply(highlight_below_75, subset=pd.IndexSlice[:, :'CV Accuracy']).format("{:.2%}", subset=pd.IndexSlice[:, :'CV Accuracy'])
styled_df

# 결론

## 고객 이탈 예측 분석
LightGBM은 91%의 가장 높은 Attrited Customer Recall과 89%의 정밀도를 가지고 있음
고객 이탈을 사전에 방지하기 위해서 LightGBM 모델을 사용하는 것이 적합함